In [7]:
import yaml
import pickle

with open("default_complete.yaml", 'r') as run_config: 
    data_config = yaml.load(run_config) 
with open("data/database_ml_parameters.yml", 'r') as param_config: 
    data_param = yaml.load(param_config) 
with open("data/config_model_parameters.yml", 'r') as mod_config: 
    data_model = yaml.load(mod_config) 
mcordata = "data"
indexp = 0
case = data_config["case"]
param_case = data_param[case]

In [8]:
from machine_learning_hep.listfiles import list_files_dir_lev2, list_files_lev2 
import multiprocessing as mp
import uproot

class Processer:
    # Class Attribute
    species = 'processer'

    # Initializer / Instance Attributes
    def __init__(self, datap, mcordata, indexp):

        self.n_root = datap["files_names"]["namefile_unmerged_tree"]
        self.n_reco = datap["files_names"]["namefile_reco"]
        self.n_recosk = datap["files_names"]["namefile_reco_skim"]
        self.n_treereco = datap["files_names"]["treeoriginreco"]
        
        self.v_all = datap["variables"]["var_all"]
        
        self.s_reco_unp = datap["skimming_sel"]
        self.s_reco_skim = datap["skimming2_sel"]
        
        self.d_root = datap["inputs"][mcordata]["unmerged_tree_dir"][indexp]
        self.d_reco = datap["output_folders"]["pkl_out"][mcordata][indexp]
        self.d_recosk = datap["output_folders"]["pkl_skimmed"][mcordata][indexp]
        
        self.l_root = None
        self.l_reco = None
        self.l_recosk = None
        self.maxperchunk = 30
        self.indexsample = None
        
    def set_maxperchunk(self, maxperchunk):
        self.maxperchunk = maxperchunk
    
    def buildlistreco(self):
        self.l_root, self.l_reco = list_files_dir_lev2(self.d_root, self.d_reco, self.n_root, self.n_reco)

    def buildlistskim(self):
        _, self.l_recosk = list_files_dir_lev2(self.d_reco, self.d_recosk, self.n_reco, self.n_recosk)

    def unpack(self, file_index):
        tree = uproot.open(self.l_root[file_index])[self.n_treereco]
        df = tree.pandas.df(branches=self.v_all)
        df = df.query(self.s_reco_unp)
        df.to_pickle(self.l_reco[file_index])

#     def skim(self, filein, fileout):
#         df = pickle.load(open(filein, "rb"))
#         df = df.query(self.s_reco_skim)
#         df.to_pickle(fileout)
    
    def parallelizer(self, function, argument_list):
        chunks = [argument_list[x:x+self.maxperchunk] for x in range(0, len(argument_list), self.maxperchunk)]
        for chunk in chunks:
            pool = mp.Pool(self.maxperchunk)
            _ = [pool.apply(function,args=chunk[i]) for i in range(len(chunk))] 
            pool.close()

    def unpacker(self):
        self.buildlistreco()
        arguments = [(i,) for i in range(len(self.l_root))]
        self.parallelizer(self.unpack, arguments)

#     def skimmer(self):
#         arguments = [(self.l_pkl[i], self.l_pklsk[i]) for i in range(len(self.l_pklsk))]
#         self.parallelizer(self.skim, arguments)

        
myprocess = Processer(data_param[case], mcordata, indexp)
myprocess.unpacker()

dataroot
datapkl
filesout
